In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
directory = '/home/katya/Object_vision/train_zip/train'

In [3]:
files = os.listdir(directory) 

In [4]:
images = filter(lambda x: x.endswith('.jpg'), files) 

In [5]:
images

In [6]:
data_image = pd.DataFrame(images)

In [7]:
data_image

,0
0,apple_35.jpg
1,apple_15.jpg
2,mixed_12.jpg
3,orange_13.jpg
4,orange_2.jpg
...,...
235,orange_54.jpg
236,apple_17.jpg
237,banana_51.jpg
238,apple_26.jpg


In [8]:
xml_files = filter(lambda x: x.endswith('.xml'), files) 

In [9]:
data_xml_files = pd.DataFrame(xml_files)

In [10]:
data_xml_files = list(data_xml_files[0])

In [11]:
name = data_xml_files[0]

In [12]:
from lxml import etree

In [13]:
from xml.etree import ElementTree as et

In [14]:
annot_path = os.path.join(directory, name)

In [15]:
annot_path

'/home/katya/Object_vision/train_zip/train/orange_17.xml'

In [16]:
name

'orange_17.xml'

In [17]:
name.replace('xml', 'jpg')

'orange_17.jpg'

In [18]:
labels = []
difficult = []
i = 0
dictionary = dict()

for k in range(len(data_xml_files)):
    name = data_xml_files[k]
    tree = et.parse(os.path.join(directory, name))
    root = tree.getroot()
    # box coordinates for xml files are extracted and corrected for image size given
    for member in root.findall('object'):
        i+=1
        label = member.find('name').text

        difficult = int(member.find('difficult').text)

        # bounding box
        xmin = int(member.find('bndbox').find('xmin').text)
        xmax = int(member.find('bndbox').find('xmax').text)

        ymin = int(member.find('bndbox').find('ymin').text)
        ymax = int(member.find('bndbox').find('ymax').text)

        
        annot_path= annot_path.replace('xml', 'jpg')
        a = [annot_path, label, xmin, ymin, xmax, ymax, difficult]
        #d = dict([(i, a)])
        dictionary[i] = a
        


In [19]:
data = pd.DataFrame(dictionary)

In [20]:
data = data.T

In [21]:
data

,0,1,2,3,4,5,6
1,/home/katya/Object_vision/train_zip/train/oran...,orange,140,300,489,668,0
2,/home/katya/Object_vision/train_zip/train/oran...,orange,389,337,818,688,0
3,/home/katya/Object_vision/train_zip/train/oran...,orange,621,277,965,620,0
4,/home/katya/Object_vision/train_zip/train/oran...,orange,365,61,805,419,0
5,/home/katya/Object_vision/train_zip/train/oran...,orange,415,282,1032,902,0
...,...,...,...,...,...,...,...
461,/home/katya/Object_vision/train_zip/train/oran...,orange,315,75,458,219,0
462,/home/katya/Object_vision/train_zip/train/oran...,orange,240,127,391,269,0
463,/home/katya/Object_vision/train_zip/train/oran...,orange,176,32,309,158,0
464,/home/katya/Object_vision/train_zip/train/oran...,banana,84,43,403,155,0


In [22]:
data.rename(columns={0: "image", 1: "class", 2: "xmin", 3: "ymin", 4: "xmax", 5: "ymax", 6: "difficult"})

,image,class,xmin,ymin,xmax,ymax,difficult
1,/home/katya/Object_vision/train_zip/train/oran...,orange,140,300,489,668,0
2,/home/katya/Object_vision/train_zip/train/oran...,orange,389,337,818,688,0
3,/home/katya/Object_vision/train_zip/train/oran...,orange,621,277,965,620,0
4,/home/katya/Object_vision/train_zip/train/oran...,orange,365,61,805,419,0
5,/home/katya/Object_vision/train_zip/train/oran...,orange,415,282,1032,902,0
...,...,...,...,...,...,...,...
461,/home/katya/Object_vision/train_zip/train/oran...,orange,315,75,458,219,0
462,/home/katya/Object_vision/train_zip/train/oran...,orange,240,127,391,269,0
463,/home/katya/Object_vision/train_zip/train/oran...,orange,176,32,309,158,0
464,/home/katya/Object_vision/train_zip/train/oran...,banana,84,43,403,155,0


In [ ]:
import autogluon.core as ag
from autogluon.vision import ObjectDetector